# Grid Search hyper parameters for SRST

1. Import dependencies

In [ ]:
import random
import torch
import numpy as np
import pandas as pd
import time
import glob
import os

from src.util.torch import resolve_torch_device
from src.data.indian_pines import load_indian_pines
from src.util.hsi import sample_from_segmentation_matrix
from src.definitions import GREED_SEARCH_FOLDER
from src.model.greed_search import GreedSearch
from src.model.spatial_regulated_self_training_greed_search import (
    SpatialRegulatedSelfTrainingPipelineGreedSearchAdapter,
)

2. Prepare env

In [2]:
random_seed = 42
cpu_count = 1

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

generator = torch.Generator()
generator.manual_seed(random_seed)

In [3]:
f"Setting num_workers to {cpu_count}"

'Setting num_workers to 1'

In [4]:
f"Device is {device}"

'Device is cuda'

# Indian pines (Init step greed search)

0. Set params

In [5]:
examples_per_class = 15
epoch_seconds = int(time.time())
# epoch_seconds = 1745249041
run_name = f"indian-pines-init-greed-search-{epoch_seconds}"

1. Load dataset

In [6]:
image, labels = load_indian_pines()

In [7]:
num_classes = len(np.unique(labels))

f"Number of classes {num_classes}"

'Number of classes 17'

In [8]:
masked_labels = sample_from_segmentation_matrix(labels, examples_per_class)

2. Train model

In [9]:
adapter = SpatialRegulatedSelfTrainingPipelineGreedSearchAdapter(
    image, masked_labels, labels, num_classes, device, cpu_count, random_seed, generator
)

In [10]:
log_dir = GREED_SEARCH_FOLDER / run_name

In [11]:
search = GreedSearch(
    adapter=adapter,
    optimize_metric="f1_score",
    log_dir=log_dir,
)

In [12]:
_, best_params, best_score = search.run()

7it [4:04:15, 2093.60s/it, high=184, high_score=0.37, low=184, low_score=0.37]


In [13]:
print("Best Params:", best_params)
print("Best Score:", best_score)

Best Params: {'k_values': [18400]}
Best Score: [{'overall_accuracy': 0.1667003333568573, 'average_accuracy': 0.523614764213562, 'kappa_score': 0.03330647945404053, 'f1_score': 0.37034767866134644}]


3. Training results

In [14]:
csv_files = glob.glob(os.path.join(log_dir, "*.csv"))

report = pd.concat([pd.read_csv(f) for f in csv_files])

report.head()

,k_values,overall_accuracy,average_accuracy,kappa_score,f1_score,best_iteration,best_f1_score
0,[10600],0.168287,0.524804,0.036831,0.372851,0,0.372851
1,[15900],0.167024,0.523615,0.033311,0.370332,0,0.370332
2,[18400],0.166700,0.523615,0.033306,0.370348,0,0.370348
3,[17200],0.166683,0.523615,0.033307,0.370348,0,0.370348
4,[17900],0.166693,0.523615,0.033308,0.370348,0,0.370348


In [15]:
len(report)

7

In [16]:
report.sort_values("kappa_score", ascending=False).head()

,k_values,overall_accuracy,average_accuracy,kappa_score,f1_score,best_iteration,best_f1_score
0,[10600],0.168287,0.524804,0.036831,0.372851,0,0.372851
1,[15900],0.167024,0.523615,0.033311,0.370332,0,0.370332
4,[17900],0.166693,0.523615,0.033308,0.370348,0,0.370348
3,[17200],0.166683,0.523615,0.033307,0.370348,0,0.370348
2,[18400],0.166700,0.523615,0.033306,0.370348,0,0.370348
